In [59]:
print("Hello WOrld!!!")

Hello WOrld!!!


In [60]:
import numpy as np
import torch
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [61]:
df  = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')

df.shape

(569, 33)

In [62]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [63]:
df.drop(columns = ['id', 'Unnamed: 32'], inplace= True)

In [64]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [65]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2, random_state=42)

In [66]:
print(X_train.shape)
print(df.shape)

(455, 30)
(569, 31)


In [67]:
scaler = StandardScaler()


X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [68]:
encoder = LabelEncoder()

y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

Numpy Arrays to PyTorch Tensor

In [69]:
X_train_tensor = torch.from_numpy(X_train_scaled)
X_test_tensor = torch.from_numpy(X_test_scaled)
y_train_tensor = torch.from_numpy(y_train_encoded)
y_test_tensor = torch.from_numpy(y_test_encoded)

In [70]:
X_train_tensor

tensor([[-1.4408, -0.4353, -1.3621,  ...,  0.9320,  2.0972,  1.8865],
        [ 1.9741,  1.7330,  2.0917,  ...,  2.6989,  1.8912,  2.4978],
        [-1.4000, -1.2496, -1.3452,  ..., -0.9702,  0.5976,  0.0579],
        ...,
        [ 0.0488, -0.5550, -0.0651,  ..., -1.2390, -0.7086, -1.2715],
        [-0.0390,  0.1021, -0.0314,  ...,  1.0500,  0.4343,  1.2134],
        [-0.5486,  0.3133, -0.6035,  ..., -0.6110, -0.3345, -0.8463]],
       dtype=torch.float64)

Creating the Model

In [ ]:
class SimpleNN():
    
    def __init__(self, X):
        self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
        self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)
        
    def forward_pass(self, X):
        y = torch.matmul(X, self.weights) + self.bias
        
        y_pred = torch.sigmoid(y)
        return y_pred
        
        
    def loss_calculation(self, y_pred, y_true):
        epsilon = 1e-7
        
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
        
        loss = -(y_true*torch.log(y_pred) + (1-y_true)*torch.log(1-y_pred)).mean()
        
        return loss
    
 
        
    
        
        

Parameters

In [89]:
learning_rate = 0.05
epochs = 500

Training Pipeline

In [90]:
# creating model
model = SimpleNN(X_train_tensor)

print(model.bias)

for epoch in range(epochs):
    
    # forward pass
    
    y_pred = model.forward_pass(X_train_tensor)

    # calculate loss
    loss = model.loss_calculation(y_pred, y_train_tensor)
    print(f" for epoch : {epoch}/{epochs}, loss is {loss.item()}")
    
    # backward pass
    
    loss.backward()


    # parameters update
    with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias -= learning_rate * model.bias.grad

    # zero gradients
    model.weights.grad.zero_()
    model.bias.grad.zero_()
    

tensor([0.], dtype=torch.float64, requires_grad=True)
 for epoch : 0/500, loss is 3.6306229215516495
 for epoch : 1/500, loss is 3.5662298223307407
 for epoch : 2/500, loss is 3.5014954091290655
 for epoch : 3/500, loss is 3.436138208947765
 for epoch : 4/500, loss is 3.370685947332795
 for epoch : 5/500, loss is 3.3046441335624364
 for epoch : 6/500, loss is 3.2386742713521124
 for epoch : 7/500, loss is 3.1712739273818302
 for epoch : 8/500, loss is 3.1022161542917557
 for epoch : 9/500, loss is 3.031878380952748
 for epoch : 10/500, loss is 2.958865018694129
 for epoch : 11/500, loss is 2.8853356899423668
 for epoch : 12/500, loss is 2.8093998923495525
 for epoch : 13/500, loss is 2.7321901642064437
 for epoch : 14/500, loss is 2.6556848470181227
 for epoch : 15/500, loss is 2.578708865290636
 for epoch : 16/500, loss is 2.502721556514821
 for epoch : 17/500, loss is 2.4238778749511094
 for epoch : 18/500, loss is 2.3455019170785363
 for epoch : 19/500, loss is 2.268175914922609
 fo

In [93]:
with torch.no_grad():
    y_test_pred = model.forward_pass(X_test_tensor)
    y_test_pred_labels = (y_test_pred >= 0.5).float()
    
    accuracy = (y_test_pred_labels.squeeze() == y_test_tensor).float().mean()
    
    print(f"Test Accuracy: {accuracy.item()*100:.2f}%")

Test Accuracy: 63.16%
